## 2. Banco de Dados

### Descrição

- Criação de um banco de dados contendo as informações dos arquivos csv
- O banco de dados será criado com a engine SQLite
- Serão criadas funções para criação, conexão e leitura do banco de dados
- Ao final, haverá uma função responsável por criar todo o banco de dados que será utlizado na aplicação  

### Criando banco de dados

In [1]:
#Carregando bibliotecas
import sys
#Adicionando diretório dos módulos
sys.path.append('../dashboard')

from os import listdir, path, remove
import sqlite3
from bdconfig import carrega_csv
import pandas as pd

In [2]:
#Carregando arquivo csv
arquivo = '../dados/JoaoPessoa_clima.csv'

df = carrega_csv(arquivo)
df.head()

,TempBulboSeco,TempBulboUmido,UmidadeRelativa,PressaoAtmEstacao,DirecaoVento,VelocidadeVento,Nebulosidade
Horario,,,,,,,
2008-01-01 00:00:00,26.8,22.6,68.0,1012.1,Sudeste,3.3,3.0
2008-01-01 12:00:00,29.4,23.3,58.0,1012.7,Sudeste,3.5,4.0
2008-01-01 18:00:00,29.6,23.5,59.0,1011.2,Sudeste,3.3,3.0
2008-01-02 00:00:00,26.6,22.2,69.0,1012.8,Sudeste,2.6,4.0
2008-01-02 12:00:00,29.8,23.9,61.0,1013.3,Sudeste,3.3,3.0


### __Criando Banco de Dados SQLite__

In [3]:
#função para criar banco de dados
def cria_bd(bd_file):

    #bd_file: nome do arquivo

    #testa se o arquvio existe    
    if not path.exists(bd_file):
        con = None
        try:
            con = sqlite3.connect(bd_file)
        except sqlite3.Error as e:
            print(e)
        finally:
            if con:
                con.close()
    else:
        print("Arquivo existe")

In [4]:
#Criando banco de dados de teste
bd_teste='../dados/teste.db'
cria_bd(bd_teste)

In [5]:
#Confirmando que o banco de dados foi criado
listdir('../dados/')
'teste.db' in listdir('../dados/')

True

In [6]:
#Apagando banco de dadosb
remove('../dados/teste.db')

### __Criando Banco de Dados Completo__

In [8]:
def bd_clima():
    path = "../dados/"

    if "clima.db" in os.listdir(path):
        print("Banco de dados já criado")
    else:
        con = None
        try:
            
            con = sqlite3.connect(path+"clima.db")
            
            #Loop dos arquivos csv
            arquivos = [arq for arq in os.listdir(path) if '.csv' in arq]

            for tabela in arquivos:

                if '_clima.csv' in tabela:
                    nome= tabela.rsplit("_clima.csv")
                    df = carrega_csv(path+tabela)
                    df.to_sql(nome[0], con)
                
                else:
                    nome = tabela.rsplit(".csv")
                    df = carrega_csv(path+tabela)
                    df.to_sql(nome[0], con)

            print("Banco de dados criado")
       
        except sqlite3.Error as e:
            print(e)
        
        finally:
            if con:
                con.close()

In [10]:
bd_clima()

Banco de dados criado


### __Consultando Banco de Dados com Pandas__

In [19]:
bd_file = '../dados/clima.db'
#Criando conexão com banco de dados
con = sqlite3.connect(bd_file)

In [12]:
#Criando query para a tabela de João Pessoa
query = "SELECT * from JoaoPessoa"

In [14]:
#Executando query com pandas
df_jp = pd.read_sql_query(query, con=con, parse_dates='Horario', index_col='Horario')
df_jp.head(10)

,TempBulboSeco,TempBulboUmido,UmidadeRelativa,PressaoAtmEstacao,DirecaoVento,VelocidadeVento,Nebulosidade
Horario,,,,,,,
2008-01-01 00:00:00,26.8,22.6,68.0,1012.1,Sudeste,3.3,3.0
2008-01-01 12:00:00,29.4,23.3,58.0,1012.7,Sudeste,3.5,4.0
2008-01-01 18:00:00,29.6,23.5,59.0,1011.2,Sudeste,3.3,3.0
2008-01-02 00:00:00,26.6,22.2,69.0,1012.8,Sudeste,2.6,4.0
2008-01-02 12:00:00,29.8,23.9,61.0,1013.3,Sudeste,3.3,3.0
2008-01-02 18:00:00,29.6,24.1,63.0,1010.8,Sudeste,4.6,3.0
2008-01-03 00:00:00,27.0,23.9,77.0,1013.8,Sudeste,3.3,5.0
2008-01-03 12:00:00,29.8,24.3,64.0,1014.5,Sudeste,6.0,4.0
2008-01-03 18:00:00,29.8,24.7,66.0,1012.4,Sul,5.6,4.0


In [15]:
df_jp.dtypes

TempBulboSeco        float64
TempBulboUmido       float64
UmidadeRelativa      float64
PressaoAtmEstacao    float64
DirecaoVento          object
VelocidadeVento      float64
Nebulosidade         float64
dtype: object

In [16]:
df_jp.index

DatetimeIndex(['2008-01-01 00:00:00', '2008-01-01 12:00:00',
               '2008-01-01 18:00:00', '2008-01-02 00:00:00',
               '2008-01-02 12:00:00', '2008-01-02 18:00:00',
               '2008-01-03 00:00:00', '2008-01-03 12:00:00',
               '2008-01-03 18:00:00', '2008-01-04 00:00:00',
               ...
               '2018-12-28 18:00:00', '2018-12-29 00:00:00',
               '2018-12-29 12:00:00', '2018-12-29 18:00:00',
               '2018-12-30 00:00:00', '2018-12-30 12:00:00',
               '2018-12-30 18:00:00', '2018-12-31 00:00:00',
               '2018-12-31 12:00:00', '2018-12-31 18:00:00'],
              dtype='datetime64[ns]', name='Horario', length=11938, freq=None)

In [20]:
#Carregando tabela de informações
query_info = "SELECT * FROM Info"
info = pd.read_sql_query(query_info, con=con)
info

,index,Cidade,Latitude,Longitude,Altitude
0,0,Areia,-6.97,-35.68,574.62
1,1,Campina-Grande,-7.22,-35.88,547.56
2,2,Joao-Pessoa,-7.10,-34.86,7.43
3,3,Monteiro,-7.88,-37.06,603.66
4,4,Patos,-7.01,-37.26,249.09
5,5,Sao-Goncalo,-6.75,-38.21,233.06


In [21]:
#Fechando conexão com banco de dados
con.close()

#### __Criando Função para Carregar Tabela__

In [22]:
def carrega_tabela(nome, bd, clima=False):

    #nome: Nome da tabela
    #bd: Arquivo do banco de dados 
    #Clima: Define se a tabela é de informações ou medições metereológicas 
    
    con = None
    try:

        con = sqlite3.connect(bd)
        query = "SELECT * FROM {}".format(nome)
        if clima:
            df = pd.read_sql_query(query, con=con, parse_dates='Horario', index_col='Horario')
        else:
            df = pd.read_sql_query(query, con=con)
            df.drop(columns='index', inplace=True)
    
    except sqlite3.Error as e:
            print(e)
        
    finally:
        if con:
            con.close()
    
    return df

In [23]:
df_tabela = carrega_tabela("JoaoPessoa", bd_file, clima=True)
df_tabela.head()

,TempBulboSeco,TempBulboUmido,UmidadeRelativa,PressaoAtmEstacao,DirecaoVento,VelocidadeVento,Nebulosidade
Horario,,,,,,,
2008-01-01 00:00:00,26.8,22.6,68.0,1012.1,Sudeste,3.3,3.0
2008-01-01 12:00:00,29.4,23.3,58.0,1012.7,Sudeste,3.5,4.0
2008-01-01 18:00:00,29.6,23.5,59.0,1011.2,Sudeste,3.3,3.0
2008-01-02 00:00:00,26.6,22.2,69.0,1012.8,Sudeste,2.6,4.0
2008-01-02 12:00:00,29.8,23.9,61.0,1013.3,Sudeste,3.3,3.0


In [25]:
df = carrega_tabela("Info", bd_file, clima=False)
df.head()

,Cidade,Latitude,Longitude,Altitude
0,Areia,-6.97,-35.68,574.62
1,Campina-Grande,-7.22,-35.88,547.56
2,Joao-Pessoa,-7.10,-34.86,7.43
3,Monteiro,-7.88,-37.06,603.66
4,Patos,-7.01,-37.26,249.09


In [27]:
df = carrega_tabela("Areia", bd_file, clima=True)
df.head()

,TempBulboSeco,TempBulboUmido,UmidadeRelativa,PressaoAtmEstacao,DirecaoVento,VelocidadeVento,Nebulosidade
Horario,,,,,,,
2008-01-01 00:00:00,26.8,22.6,68.0,1012.1,Sudeste,3.3,3.0
2008-01-01 12:00:00,29.4,23.3,58.0,1012.7,Sudeste,3.5,4.0
2008-01-01 18:00:00,29.6,23.5,59.0,1011.2,Sudeste,3.3,3.0
2008-01-02 00:00:00,26.6,22.2,69.0,1012.8,Sudeste,2.6,4.0
2008-01-02 12:00:00,29.8,23.9,61.0,1013.3,Sudeste,3.3,3.0


In [28]:
df = carrega_tabela("SaoGoncalo", bd_file, clima=True)
df.head()

,TempBulboSeco,TempBulboUmido,UmidadeRelativa,PressaoAtmEstacao,DirecaoVento,VelocidadeVento,Nebulosidade
Horario,,,,,,,
2008-01-01 00:00:00,26.8,22.6,68.0,1012.1,Sudeste,3.3,3.0
2008-01-01 12:00:00,29.4,23.3,58.0,1012.7,Sudeste,3.5,4.0
2008-01-01 18:00:00,29.6,23.5,59.0,1011.2,Sudeste,3.3,3.0
2008-01-02 00:00:00,26.6,22.2,69.0,1012.8,Sudeste,2.6,4.0
2008-01-02 12:00:00,29.8,23.9,61.0,1013.3,Sudeste,3.3,3.0


In [29]:
df = carrega_tabela("Monteiro", bd_file, clima=True)
df.head()

,TempBulboSeco,TempBulboUmido,UmidadeRelativa,PressaoAtmEstacao,DirecaoVento,VelocidadeVento,Nebulosidade
Horario,,,,,,,
2008-01-01 00:00:00,26.8,22.6,68.0,1012.1,Sudeste,3.3,3.0
2008-01-01 12:00:00,29.4,23.3,58.0,1012.7,Sudeste,3.5,4.0
2008-01-01 18:00:00,29.6,23.5,59.0,1011.2,Sudeste,3.3,3.0
2008-01-02 00:00:00,26.6,22.2,69.0,1012.8,Sudeste,2.6,4.0
2008-01-02 12:00:00,29.8,23.9,61.0,1013.3,Sudeste,3.3,3.0


In [30]:
df = carrega_tabela("CampinaGrande", bd_file, clima=True)
df.head()

,TempBulboSeco,TempBulboUmido,UmidadeRelativa,PressaoAtmEstacao,DirecaoVento,VelocidadeVento,Nebulosidade
Horario,,,,,,,
2008-01-01 00:00:00,26.8,22.6,68.0,1012.1,Sudeste,3.3,3.0
2008-01-01 12:00:00,29.4,23.3,58.0,1012.7,Sudeste,3.5,4.0
2008-01-01 18:00:00,29.6,23.5,59.0,1011.2,Sudeste,3.3,3.0
2008-01-02 00:00:00,26.6,22.2,69.0,1012.8,Sudeste,2.6,4.0
2008-01-02 12:00:00,29.8,23.9,61.0,1013.3,Sudeste,3.3,3.0


In [31]:
df = carrega_tabela("Patos", bd_file, clima=True)
df.head()

,TempBulboSeco,TempBulboUmido,UmidadeRelativa,PressaoAtmEstacao,DirecaoVento,VelocidadeVento,Nebulosidade
Horario,,,,,,,
2008-01-01 00:00:00,26.8,22.6,68.0,1012.1,Sudeste,3.3,3.0
2008-01-01 12:00:00,29.4,23.3,58.0,1012.7,Sudeste,3.5,4.0
2008-01-01 18:00:00,29.6,23.5,59.0,1011.2,Sudeste,3.3,3.0
2008-01-02 00:00:00,26.6,22.2,69.0,1012.8,Sudeste,2.6,4.0
2008-01-02 12:00:00,29.8,23.9,61.0,1013.3,Sudeste,3.3,3.0
